--- Day 13: Packet Scanners ---
You need to cross a vast firewall. The firewall consists of several layers, each with a security scanner that moves back and forth across the layer. To succeed, you must not be detected by a scanner.

By studying the firewall briefly, you are able to record (in your puzzle input) the depth of each layer and the range of the scanning area for the scanner within it, written as depth: range. Each layer has a thickness of exactly 1. A layer at depth 0 begins immediately inside the firewall; a layer at depth 1 would start immediately after that.

For example, suppose you've recorded the following:

0: 3
1: 2
4: 4
6: 4
This means that there is a layer immediately inside the firewall (with range 3), a second layer immediately after that (with range 2), a third layer which begins at depth 4 (with range 4), and a fourth layer which begins at depth 6 (also with range 4). Visually, it might look like this:

 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]
Within each layer, a security scanner moves back and forth within its range. Each security scanner starts at the top and moves down until it reaches the bottom, then moves up until it reaches the top, and repeats. A security scanner takes one picosecond to move one step. Drawing scanners as S, the first few picoseconds look like this:


Picosecond 0:
 0   1   2   3   4   5   6
[S] [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

Picosecond 1:
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

Picosecond 2:
 0   1   2   3   4   5   6
[ ] [S] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

Picosecond 3:
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]
Your plan is to hitch a ride on a packet about to move through the firewall. The packet will travel along the top of each layer, and it moves at one layer per picosecond. Each picosecond, the packet moves one layer forward (its first move takes it into layer 0), and then the scanners move one step. If there is a scanner at the top of the layer as your packet enters it, you are caught. (If a scanner moves into the top of its layer while you are there, you are not caught: it doesn't have time to notice you before you leave.) If you were to do this in the configuration above, marking your current position with parentheses, your passage through the firewall would look like this:

Initial state:
 0   1   2   3   4   5   6
[S] [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

Picosecond 0:
 0   1   2   3   4   5   6
(S) [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
( ) [ ] ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]


Picosecond 1:
 0   1   2   3   4   5   6
[ ] ( ) ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] (S) ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]


Picosecond 2:
 0   1   2   3   4   5   6
[ ] [S] (.) ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] (.) ... [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]


Picosecond 3:
 0   1   2   3   4   5   6
[ ] [ ] ... (.) [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]

 0   1   2   3   4   5   6
[S] [S] ... (.) [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]


Picosecond 4:
 0   1   2   3   4   5   6
[S] [S] ... ... ( ) ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... ( ) ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]


Picosecond 5:
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] (.) [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [S] ... ... [S] (.) [S]
[ ] [ ]         [ ]     [ ]
[S]             [ ]     [ ]
                [ ]     [ ]


Picosecond 6:
 0   1   2   3   4   5   6
[ ] [S] ... ... [S] ... (S)
[ ] [ ]         [ ]     [ ]
[S]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... ( )
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]
                
In this situation, you are caught in layers 0 and 6, because your packet entered the layer when its scanner was at the top when you entered it. You are not caught in layer 1, since the scanner moved into the top of the layer once you were already there.

The severity of getting caught on a layer is equal to its depth multiplied by its range. (Ignore layers in which you do not get caught.) The severity of the whole trip is the sum of these values. In the example above, the trip severity is 0*3 + 6*4 = 24.

Given the details of the firewall you've recorded, if you leave immediately, what is the severity of your whole trip?

In [3]:
from math import *

class Wall(object):    
    def __init__(self, height, depth, sentry=True):
        self.height = height
        self.depth = depth
        self.sentry = sentry

        self.severity = height * depth
        self.clock = Wall.gen(height)
        if self.sentry:
            self.pos = next(self.clock)
        else:
            self.pos = -1
            
    def __repr__(self):
        ret = "{}: {} -> {}".format(self.depth, self.height, self.pos)
        return ret
    
    def visual(self, lvl):
        ret = ""
        if self.sentry and self.height > lvl:
            if self.is_sentry(lvl):
                ret = "[S]"
            else:
                ret = "[ ]"
        else:
            ret = "..."
        return ret
    
    def tick(self):
        if self.sentry:
            self.pos = next(self.clock)
            
    def is_sentry(self, lvl):
        if self.pos == lvl:
            return True
        return False
        
    @classmethod
    def gen(cls, height):
        while True:
            for i in range(height-1):
                yield(i)
            for j in range(height-1, 0, -1):
                yield(j)
                        
            
class Security(object):
    def __init__(self):
        self.walls = []
        self.sneak = 0
        self.max_depth = 0
     
    def tick(self):
        for w in self.walls:
            w.tick()
            
    def parse(self, inpt):
        self.depth = max(inpt.keys())+1
        self.height = max(inpt.values())
        for tick in range(self.depth):
            if tick in inpt:
                self.walls.append(Wall(inpt[tick], tick))
            else:
                self.walls.append(Wall(0, tick, False))
                
    def visual(self):
        for j in range(self.height):
            print(" ".join([x.visual(j) for x in self.walls]))
        print(" ".join([str(x.pos) for x in self.walls]))
    
    def step(self, move=True):
        self.tick()
        if move:
            self.sneak += 1
        
    def is_sentry(self, depth):
        return self.walls[depth].is_sentry(0)
    
    def delay(self, delay):
        for j in range(delay):
            self.step(move=False)
        
        
    def traverse(self):
        total_cost = 0
        self.visual()
        for i in range(self.depth):            
            gotcha = self.is_sentry(i)
            if gotcha:
                return True
            self.step()
        return False
            
        
                    

In [ ]:
test = {
    0: 3,
    1: 2,
    4: 4,
    6: 4,
    8: 2,
    10: 20
}

# data = test
m_iter = max(data.values())*100

gotcha = False
for i in range(2300, 2300*2):
    print("Delay: {}".format(i))
    sec = Security()
    sec.parse(data)
    sec.delay(i)
    gotcha = sec.traverse()
    if not gotcha:
        print("Success")
        break
        
print("{} iterations".format(i))


Delay: 2300
[ ] [S] [S] ... [ ] ... [ ] ... [S] ... [ ] ... [S] ... [S] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [S] ... [ ] ... ... ... ... ... [ ]
[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] 

[ ] [S] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] ... [S] ... [S] ... [ ] ... [S] ... [S] ... [S] ... [S] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [S] ... 

[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] [S] [S] ... [S] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [S] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... 

[ ] [S] [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] ... [ ] ... [ ] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [ ] ... [ ] ... [S] ... [S] ... [S] ... [S] ... [ ] ... [S] ... [S] ... [ ] ... 

[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] [S] [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [ ] ... [ ] ... [S] ... [S] ... [S] ... [S] ... [ ] ... [S] ... [S] ... [ ] ... [ ] ... ... ... [S] ... [ ] ... ... ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... 

...

[ ] [S] [S] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... 

[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] [S] [S] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [S] ... [ ] ... ... ... ... ... [ ]
[ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... 

[ ] [S] [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [S] ... [S] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] ... [S] ... [S] ... [S] ... [ ] ... [S] ... [ ] ... [ ] ... [S] ... [S] ... [ ] ... [S] ... [S] ... [S] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... 

[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] [S] [S] ... [S] ... [S] ... [ ] ... [S] ... [ ] ... [ ] ... [S] ... [S] ... [ ] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [ ] ... [S] ... [S] ... [S] ... [S] ... [ ] ... [S] ... [S] ... [ ] ... [ ] ... ... ... [S] ... [ ] ... ... ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... 

[ ] [S] [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [S] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[ ] [ ] [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... ... ... [ ] ... [ ] ... ... ... ... ... ... ... [ ] ... [ ] ... ... ... ... ... [ ]
[S] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [ ] ... [S] ... [S] ... [ ] ... [ ] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... [S] ... 

In [39]:
[(x, ((x-1)*2)) for x in range(2, 10)]

[(2, 2), (3, 4), (4, 6), (5, 8), (6, 10), (7, 12), (8, 14), (9, 16)]

In [110]:
class Layer(object):
    def __init__(self, height, depth, sentry=True):
        self.height = height
        self.depth = depth
        self.sentry = sentry
        self.rotation = (self.height-1)*2
        
        
    def __repr__(self):
        return "depth: {}, height: {}, rotation: {}".format(self.depth, self.height, self.rotation)
    
    def is_caught(self, tick):
        if self.sentry:
            return not bool(tick % self.rotation)
        return False
    
    def cost(self, tick):
        if self.is_caught(tick):
            print("tick {} is caught at {}".format(tick, self))
            return self.height*self.depth
        return 0
    
class Defenses(object):
    def __init__(self):
        self.layers = []
    
    def cost(self, offset=0):
        cost = 0
        for i in range(len(self.layers)):
            tick = i + offset
            cost += self.layers[i].cost(tick)
        return cost
    
    def is_caught(self, offset=0):
        for i in range(len(self.layers)):
#             print("layer: {}, tick: {}".format(i, offset+i))
            if self.layers[i].is_caught(offset+i):
                return True
        print("Success at offset: {}".format(offset))
        return False
          
    @staticmethod
    def parse(inpt):
        ret = Defenses()
        for depth in range(max(inpt.keys())+1):            
            if depth in inpt:
                height = inpt[depth]
                ret.layers.append(Layer(height, depth))
            else:
                ret.layers.append(Layer(-1,depth,sentry=False))
        return ret
                 

In [111]:
inpt = {
    0: 3,
    1: 2,
    4: 4,
    6: 4,
}

In [124]:
defs = Defenses.parse(data)

In [128]:
for i in range(len(data.keys())*100000):
    if not defs.is_caught(offset=i):
        print("Success at {}".format(i))

Success at offset: 3830344
Success at 3830344


In [35]:
one = defs.layers[0]

In [38]:
one.is_caught(1)

False

In [114]:
inpt = """
0: 4
1: 2
2: 3
4: 5
6: 8
8: 6
10: 4
12: 6
14: 6
16: 8
18: 8
20: 6
22: 8
24: 8
26: 8
28: 12
30: 12
32: 9
34: 14
36: 12
38: 12
40: 12
42: 12
44: 10
46: 12
48: 12
50: 10
52: 14
56: 12
58: 14
62: 14
64: 14
66: 12
68: 14
70: 14
72: 17
74: 14
76: 14
80: 20
82: 14
90: 24
92: 14
98: 14
"""
data = {}
for line in inpt.split('\n'):
    if line:
        key, value = line.split(':')
        data[int(key)] = int(value)
        
len(data), data[98]

(43, 14)